In [1]:
import os
import json
from multiprocessing import Pool
from datetime import datetime
import pandas as pd

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import shapely.geometry
from colour import Color

In [2]:
data_folder = "./output"
output_folder = "./frames"
geo_shapes_folder = "./geodata"
os.makedirs(output_folder, exist_ok=True)

In [3]:
#fnames = [os.path.join(root, f) for root, _, files in os.walk(geo_shapes_folder) for f in files if f.endswith(())]
#local_data = [list(shpreader.Reader(shape_file).geometries()) for shape_file in fnames]
country_color = "#dbdbdb"
urban_areas_color = "#c2c2c2"
text_color = "black"
border_color = "black"
czech_rep_country_color = "#cecece"
czech_rep_country_color = "#6baf78"

geo_resources = [
    #(
    #    shpreader.natural_earth(resolution = "10m", category='cultural', name="urban_areas"),
    #    {"edgecolor": 'none', "facecolor": urban_areas_color, 'alpha': 0.5, 'zorder': 2}
    #),
]

countries = (
        [c.geometry for c in shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="admin_1_states_provinces")).records() if c.attributes['admin'] != 'Czech Republic'],
        {"edgecolor": border_color, "facecolor": country_color, 'alpha': 1, 'zorder': 1}
        )

czech_rep = (
        [c.geometry for c in shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="admin_1_states_provinces")).records() if c.attributes['admin'] == 'Czech Republic'],
        {"edgecolor": border_color, "facecolor": czech_rep_country_color, 'alpha': 1, 'zorder': 1}
)

data = [(shpreader.Reader(resource).geometries(), styling) for resource, styling in geo_resources] + [countries] + [czech_rep]
cities = [city for city in shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="populated_places")).records() if city.attributes.get('TIMEZONE') and city.attributes['TIMEZONE'].startswith('Europe')]
roads = list(shpreader.Reader(shpreader.natural_earth(resolution = "10m", category='cultural', name="roads")).geometries())
with open('./zones.json', 'r') as zf:
        zones = json.load(zf)

In [22]:

def render_frame(input_filepath: str, output_filepath: str):
    try:
        with open(input_filepath, "r", encoding="utf-8-sig") as infile:
            vehicles_json = json.load(infile)
    except Exception as e:
        print(f"{e} {input_filepath} -> {output_filepath}")
        return

    vehicles = vehicles_json["Vehicles"]
    vehicles_df = pd.DataFrame(vehicles)
    try:
        active_vehicles_df = vehicles_df[vehicles_df.IsInactive==False]
    except Exception as e:
        print(f"{e} {input_filepath} -> {output_filepath}")
        active_vehicles_df = vehicles_df

    timestamp_string_formatted = vehicles_json["LastUpdate"][:19]
    timestamp = datetime.strptime(timestamp_string_formatted, "%Y-%m-%dT%H:%M:%S")
    
    fig = plt.figure(figsize=(10,5), facecolor='white', tight_layout=True)
    ax = plt.subplot(projection=ccrs.PlateCarree())
    #plt.title('Aktivní vozidla MHD IDSJMK\n', loc='center', size=28,fontweight="bold")
    plt.title(timestamp.strftime("%A - %Y-%m-%d %H:%M:%S"), loc='center', size=28,fontweight="bold")
    plt.tight_layout()

    # render background
    for geometry, styling in data:
       ax.add_geometries(geometry, ccrs.PlateCarree(), **styling)

    # render transit zones
    for zone in zones:
        zone_polygon = shapely.geometry.Polygon([pos[::-1] for pos in zone["Coordinates"]])
        ax.add_geometries([zone_polygon], ccrs.PlateCarree(), facecolor="none",edgecolor="black", zorder=2, alpha=0.05)
        #ax.annotate(f"{zone['Name']}", zone['Center'][::-1], zorder = 3, alpha=0.05)

    # render cities
    ax.scatter([c.geometry.x for c in cities],[c.geometry.y for c in cities], c="black", s=10, alpha=1, zorder=999, marker='s')
    for city in cities:
        pos = city.geometry
        ax.annotate(city.attributes['NAME'], (pos.x+0.01, pos.y+0.01), zorder=999)

    delay_value_range = abs(active_vehicles_df['Delay'].max()) + 1
    delay_colour_map = list(Color("#000088").range_to(Color("red"), delay_value_range))
    
    # render vehicles
    for v in active_vehicles_df.itertuples():
        color_index = max(v.Delay, 0)
        ax.annotate(f"({v.Delay})", (v.Lng, v.Lat), fontsize=11,
        #c=colorFader("#000088", "red", max((v.Delay-min_delay)/(max_delay-min_delay), 0)),
        zorder = max(v.Delay, 0) + 4,
        c=delay_colour_map[color_index].hex)

    #if not active_vehicles_df.empty:
    #   ax.scatter(active_vehicles_df['Lng'], active_vehicles_df['Lat'], c="blue", s=1, alpha=1, zorder=5)

    lat_min = 48.6
    lat_max = 49.65
    lng_min = 15.5
    lng_max = 17.7
    ax.set_extent([lng_min, lng_max, lat_min, lat_max], crs=ccrs.PlateCarree())
    plt.savefig(output_filepath, dpi=80)
    plt.close(fig)

In [23]:
input_filepaths = sorted([os.path.join(data_folder,f) for f in os.listdir(data_folder) if os.path.isfile(os.path.join(data_folder, f))])
params = [(infile, os.path.join(output_folder,f"{i:06d}.png")) for i, infile in enumerate(input_filepaths)]

render_frame(params[500][0], params[500][1])

#with Pool(maxtasksperchild=100) as p:
#    p.starmap(render_frame, params)
